# Housing Prices Competition for Kaggle Learn Users

Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

The Ames Housing dataset was compiled by Dean De Cock for use in data science education. It's an incredible alternative for data scientists looking for a modernized and expanded version of the often cited Boston Housing dataset. 

### Import all necessary libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder
import numpy as np

# Data Monitoring and Pre-processing

### Loading the Data

In [39]:
df_train = pd.read_csv(r"home-data-for-ml-course/train.csv")
df_test = pd.read_csv(r"home-data-for-ml-course/test.csv")

In [40]:
# # df_train = df_train.replace("NaN", " ", regex=True)
# df_train = df_train.fillna((0))
# # df_test = df_test.replace("NaN", " ", regex=True)
# df_test = df_test.fillna(float(0))

### Splitting the Data into input and output

In [40]:
X_train = df_train.drop(["Id", "SalePrice"], axis=1)
y_train = df_train.SalePrice

X_test = df_test.drop(["Id"], axis=1) # test data

print("Train: ", X_train.shape)
print("Test: ", X_test.shape)

Train:  (1460, 79)
Test:  (1459, 79)


### Identifyng object items

In [41]:
train_object_list = []
test_object_list = []
for index, value in enumerate(X_train.dtypes):
    if value == "O":
        train_object_list.append(X_train.dtypes.index[index])

for index, value in enumerate(X_test.dtypes):
    if value == "O":
        test_object_list.append(X_test.dtypes.index[index])

### Object item Dataframe

In [42]:
train_obj_df = pd.DataFrame(data= X_train[train_object_list])
test_obj_df = pd.DataFrame(data= X_test[test_object_list])

### converting NaN values to string 0 at object DataFrame

In [43]:
train_obj_df = train_obj_df.fillna(str(0))
test_obj_df = test_obj_df.fillna(str(0))

### Identifyng the mixed type items from object items

In [44]:
train_mixed_val = []
test_mixed_val = []
for i in train_object_list:
    for j in range(len(X_train[i])):
        if type(X_train[i][j]) == float:
            train_mixed_val.append(i)
            break
            
for i in test_object_list:
    for j in range(len(X_test[i])):
        if type(X_test[i][j]) == float:
            test_mixed_val.append(i)
            break

In [45]:
train_obj_df.shape

(1460, 43)

In [46]:
len(train_mixed_val)

16

### Converting float to str at object DataFrame

In [47]:
for i in train_mixed_val:
    for j in range(len(train_obj_df[i])):
        if type(train_obj_df[i][j]) == float:
            train_obj_df[i][j] = str(train_obj_df[i][j])
        elif type(train_obj_df[i][j]) == int:
            train_obj_df[i][j] = str(train_obj_df[i][j])
        else:
            pass

for i in test_mixed_val:
    for j in range(len(test_obj_df[i])):
        if type(test_obj_df[i][j]) == float:
            test_obj_df[i][j] = str(test_obj_df[i][j])
        elif type(test_obj_df[i][j]) == int:
            test_obj_df[i][j] = str(test_obj_df[i][j])
        else:
            pass

### Dropping the object items

In [48]:
X_train = X_train.drop(train_object_list, axis=1)
X_test = X_test.drop(test_object_list, axis=1)

In [49]:
print("Train: ", X_train.shape)
print("Test: ", X_test.shape)

Train:  (1460, 36)
Test:  (1459, 36)


### Converting NaN values of X DataFrame

In [50]:
X_train = X_train.fillna(float(0))
X_test = X_test.fillna(float(0))

### Data Encoding

In [51]:
class Encoding:
    def ordinal_encoder(self, train, test):
        oe = OrdinalEncoder()
        X_train_enc = oe.fit_transform(train)
        X_test_enc = oe.fit_transform(test)
        
        return X_train_enc, X_test_enc
    
    def one_hot_encoder(self, train, test):
        ohe = OneHotEncoder()
        
        X_train_enc = ohe.fit_transform(train)
        X_test_enc = ohe.fit_transform(test)
        
        return X_train_enc, X_test_enc
    
    def label_encoder(self, train):
        le = LabelEncoder()
        y_train_enc = le.fit_transform(train)
#         X_test_enc = le.fit_transform(test)
        
        return y_train_enc

In [52]:
encode = Encoding()

### Encoding the remained X dataFrame

In [16]:
X_train_enc, X_test_enc = encode.ordinal_encoder(X_train, X_test)

### Encoding the object DataFrame

In [53]:
tr_ob_df, te_ob_df = encode.ordinal_encoder(train_obj_df, test_obj_df)

### Get back to the DataFrame again after encoding

In [54]:
train_obj_df = pd.DataFrame(data= tr_ob_df, columns= train_obj_df.columns)
test_obj_df = pd.DataFrame(data= te_ob_df, columns= test_obj_df.columns)

In [19]:
X_train = pd.DataFrame(data= X_train_enc, columns= X_train.columns)
X_test = pd.DataFrame(data= X_test_enc, columns= X_test.columns)

### Concate these two DataFrame

In [55]:
final_train = pd.concat([X_train, train_obj_df], axis=1)
final_test = pd.concat([X_test, test_obj_df], axis=1)

In [56]:
final_train.shape, final_test.shape

((1460, 79), (1459, 79))

# Train The Nueral Network Model

### Model Specifyng

In [94]:
class myCallback(tf.keras.callbacks.Callback):
    def epoch_end(self, epoch, logs={}):
        if (logs.get("loss") < 5.000):
            print("Reached 99.9% accuracy so cancelling training!")
            self.model.stop_running = True

callback = myCallback()


model = tf.keras.models.Sequential([tf.keras.layers.Dense(units=81, activation="relu", input_shape=[final_train.shape[1]]),
                                    tf.keras.layers.Dense(units=64, activation="relu"),
#                                     tf.keras.layers.Dense(units=32, activation="relu"),
#                                     tf.keras.layers.Dense(units=16, activation="relu"),
#                                     tf.keras.layers.Dense(units=78, activation="relu"),
                                    tf.keras.layers.Dense(units=1)])
    
model.compile(loss="mae",optimizer='adam', metrics=["accuracy"])
history = model.fit(final_train, 
                    y_train,
                    epochs = 500,
                    batch_size= 1,
                    validation_split = 0.2,
                    callbacks = [callback])

Epoch 1/500
1168/1168 [==============================] - 1s 965us/step - loss: 77155.0219 - accuracy: 0.0000e+00 - val_loss: 37178.6836 - val_accuracy: 0.0000e+00
Epoch 2/500
1168/1168 [==============================] - 1s 852us/step - loss: 32813.6888 - accuracy: 0.0000e+00 - val_loss: 31697.1035 - val_accuracy: 0.0000e+00
Epoch 3/500
1168/1168 [==============================] - 1s 855us/step - loss: 29153.7616 - accuracy: 0.0000e+00 - val_loss: 31741.1035 - val_accuracy: 0.0000e+00
Epoch 4/500
1168/1168 [==============================] - 1s 774us/step - loss: 27290.4645 - accuracy: 0.0000e+00 - val_loss: 31432.5176 - val_accuracy: 0.0000e+00
Epoch 5/500
1168/1168 [==============================] - 1s 792us/step - loss: 29096.9069 - accuracy: 0.0000e+00 - val_loss: 30132.4922 - val_accuracy: 0.0000e+00
Epoch 6/500
1168/1168 [==============================] - 1s 813us/step - loss: 26219.1180 - accuracy: 0.0000e+00 - val_loss: 31827.8691 - val_accuracy: 0.0000e+00
Epoch 7/500
1168/1168 

1168/1168 [==============================] - 1s 857us/step - loss: 20922.2762 - accuracy: 0.0000e+00 - val_loss: 26193.0215 - val_accuracy: 0.0000e+00
Epoch 51/500
1168/1168 [==============================] - 1s 832us/step - loss: 22259.6855 - accuracy: 0.0000e+00 - val_loss: 26442.4141 - val_accuracy: 0.0000e+00
Epoch 52/500
1168/1168 [==============================] - 1s 864us/step - loss: 21747.1482 - accuracy: 0.0000e+00 - val_loss: 27010.3789 - val_accuracy: 0.0000e+00
Epoch 53/500
1168/1168 [==============================] - 1s 830us/step - loss: 20900.5210 - accuracy: 0.0000e+00 - val_loss: 27998.3691 - val_accuracy: 0.0000e+00
Epoch 54/500
1168/1168 [==============================] - 1s 932us/step - loss: 22092.2506 - accuracy: 0.0000e+00 - val_loss: 29176.7871 - val_accuracy: 0.0000e+00
Epoch 55/500
1168/1168 [==============================] - 1s 834us/step - loss: 21660.8555 - accuracy: 0.0000e+00 - val_loss: 28044.1934 - val_accuracy: 0.0000e+00
Epoch 56/500
1168/1168 [=====

1168/1168 [==============================] - 1s 793us/step - loss: 19546.1204 - accuracy: 0.0000e+00 - val_loss: 25489.2637 - val_accuracy: 0.0000e+00
Epoch 98/500
1168/1168 [==============================] - 1s 807us/step - loss: 18705.2391 - accuracy: 0.0000e+00 - val_loss: 24854.5547 - val_accuracy: 0.0000e+00
Epoch 99/500
1168/1168 [==============================] - 1s 803us/step - loss: 19955.8222 - accuracy: 0.0000e+00 - val_loss: 25082.2383 - val_accuracy: 0.0000e+00
Epoch 100/500
1168/1168 [==============================] - 1s 805us/step - loss: 19448.8031 - accuracy: 0.0000e+00 - val_loss: 25046.7461 - val_accuracy: 0.0000e+00
Epoch 101/500
1168/1168 [==============================] - 1s 815us/step - loss: 19803.4893 - accuracy: 0.0000e+00 - val_loss: 25573.4648 - val_accuracy: 0.0000e+00
Epoch 102/500
1168/1168 [==============================] - 1s 883us/step - loss: 20890.4786 - accuracy: 0.0000e+00 - val_loss: 26142.0781 - val_accuracy: 0.0000e+00 21466.7525 - accu
Epoch 10

1168/1168 [==============================] - 1s 811us/step - loss: 17267.4062 - accuracy: 0.0000e+00 - val_loss: 23318.7344 - val_accuracy: 0.0000e+00
Epoch 144/500
1168/1168 [==============================] - 1s 876us/step - loss: 18821.2250 - accuracy: 0.0000e+00 - val_loss: 24576.6230 - val_accuracy: 0.0000e+00
Epoch 145/500
1168/1168 [==============================] - 1s 871us/step - loss: 18297.7281 - accuracy: 0.0000e+00 - val_loss: 26447.0918 - val_accuracy: 0.0000e+00
Epoch 146/500
1168/1168 [==============================] - 1s 881us/step - loss: 18737.8149 - accuracy: 0.0000e+00 - val_loss: 24799.4121 - val_accuracy: 0.0000e+000s - loss: 18738.0139 - accuracy: 0.0000e+
Epoch 147/500
1168/1168 [==============================] - 1s 882us/step - loss: 18422.9268 - accuracy: 0.0000e+00 - val_loss: 24504.2207 - val_accuracy: 0.0000e+0061.7989 - accuracy
Epoch 148/500
1168/1168 [==============================] - 1s 874us/step - loss: 20014.2167 - accuracy: 0.0000e+00 - val_loss: 23

1168/1168 [==============================] - 1s 801us/step - loss: 19397.6295 - accuracy: 0.0000e+00 - val_loss: 25054.3242 - val_accuracy: 0.0000e+00
Epoch 191/500
1168/1168 [==============================] - 1s 801us/step - loss: 18184.9410 - accuracy: 0.0000e+00 - val_loss: 23699.5742 - val_accuracy: 0.0000e+00
Epoch 192/500
1168/1168 [==============================] - 1s 806us/step - loss: 16607.4200 - accuracy: 0.0000e+00 - val_loss: 30990.3672 - val_accuracy: 0.0000e+00
Epoch 193/500
1168/1168 [==============================] - 1s 876us/step - loss: 18631.8197 - accuracy: 0.0000e+00 - val_loss: 25101.9434 - val_accuracy: 0.0000e+00 - loss: 19299.3084 - accuracy
Epoch 194/500
1168/1168 [==============================] - 1s 874us/step - loss: 15996.7732 - accuracy: 0.0000e+00 - val_loss: 25178.3184 - val_accuracy: 0.0000e+00
Epoch 195/500
1168/1168 [==============================] - 1s 875us/step - loss: 17769.1370 - accuracy: 0.0000e+00 - val_loss: 23562.5781 - val_accuracy: 0.000

...

Epoch 287/500
1168/1168 [==============================] - 1s 873us/step - loss: 15490.9048 - accuracy: 0.0000e+00 - val_loss: 23794.1445 - val_accuracy: 0.0000e+00
Epoch 288/500
1168/1168 [==============================] - 1s 871us/step - loss: 15854.7858 - accuracy: 0.0000e+00 - val_loss: 22865.0469 - val_accuracy: 0.0000e+00
Epoch 289/500
1168/1168 [==============================] - 1s 876us/step - loss: 14693.7001 - accuracy: 0.0000e+00 - val_loss: 22784.5059 - val_accuracy: 0.0000e+00
Epoch 290/500
1168/1168 [==============================] - 1s 872us/step - loss: 15193.3850 - accuracy: 0.0000e+00 - val_loss: 24126.1055 - val_accuracy: 0.0000e+00
Epoch 291/500
1168/1168 [==============================] - 1s 868us/step - loss: 14506.6502 - accuracy: 0.0000e+00 - val_loss: 22311.2168 - val_accuracy: 0.0000e+00
Epoch 292/500
1168/1168 [==============================] - 1s 803us/step - loss: 14896.1717 - accuracy: 0.0000e+00 - val_loss: 22862.5684 - val_accuracy: 0.0000e+00
Epoch 293/

1168/1168 [==============================] - 1s 880us/step - loss: 15688.3896 - accuracy: 0.0000e+00 - val_loss: 24383.1172 - val_accuracy: 0.0000e+00
Epoch 336/500
1168/1168 [==============================] - 1s 847us/step - loss: 14913.4686 - accuracy: 0.0000e+00 - val_loss: 22772.1465 - val_accuracy: 0.0000e+00
Epoch 337/500
1168/1168 [==============================] - 1s 819us/step - loss: 14011.2631 - accuracy: 0.0000e+00 - val_loss: 22263.9844 - val_accuracy: 0.0000e+00
Epoch 338/500
1168/1168 [==============================] - 1s 875us/step - loss: 15728.8827 - accuracy: 0.0000e+00 - val_loss: 24224.5078 - val_accuracy: 0.0000e+00
Epoch 339/500
1168/1168 [==============================] - 1s 874us/step - loss: 15115.0255 - accuracy: 0.0000e+00 - val_loss: 23843.8691 - val_accuracy: 0.0000e+00
Epoch 340/500
1168/1168 [==============================] - 1s 872us/step - loss: 14061.8345 - accuracy: 0.0000e+00 - val_loss: 22676.2910 - val_accuracy: 0.0000e+00
Epoch 341/500
1168/1168 

1168/1168 [==============================] - 1s 867us/step - loss: 14720.2392 - accuracy: 0.0000e+00 - val_loss: 22387.7676 - val_accuracy: 0.0000e+00
Epoch 385/500
1168/1168 [==============================] - 1s 806us/step - loss: 14525.1418 - accuracy: 0.0000e+00 - val_loss: 25194.1680 - val_accuracy: 0.0000e+00
Epoch 386/500
1168/1168 [==============================] - 1s 873us/step - loss: 14327.8573 - accuracy: 0.0000e+00 - val_loss: 24561.9238 - val_accuracy: 0.0000e+00
Epoch 387/500
1168/1168 [==============================] - 1s 870us/step - loss: 14530.8407 - accuracy: 0.0000e+00 - val_loss: 21892.5488 - val_accuracy: 0.0000e+00
Epoch 388/500
1168/1168 [==============================] - 1s 871us/step - loss: 14207.0511 - accuracy: 0.0000e+00 - val_loss: 22664.0059 - val_accuracy: 0.0000e+00
Epoch 389/500
1168/1168 [==============================] - 1s 880us/step - loss: 14593.8275 - accuracy: 0.0000e+00 - val_loss: 22176.8887 - val_accuracy: 0.0000e+00
Epoch 390/500
1168/1168 

1168/1168 [==============================] - 1s 827us/step - loss: 14148.0724 - accuracy: 0.0000e+00 - val_loss: 21806.7852 - val_accuracy: 0.0000e+00
Epoch 433/500
1168/1168 [==============================] - 1s 872us/step - loss: 13398.8393 - accuracy: 0.0000e+00 - val_loss: 21599.2227 - val_accuracy: 0.0000e+00
Epoch 434/500
1168/1168 [==============================] - 1s 876us/step - loss: 14293.3439 - accuracy: 0.0000e+00 - val_loss: 23057.9512 - val_accuracy: 0.0000e+00oss: 15181.0266 - ac
Epoch 435/500
1168/1168 [==============================] - 1s 887us/step - loss: 14288.6845 - accuracy: 0.0000e+00 - val_loss: 22122.9277 - val_accuracy: 0.0000e+00
Epoch 436/500
1168/1168 [==============================] - 1s 876us/step - loss: 13280.1481 - accuracy: 0.0000e+00 - val_loss: 21880.0176 - val_accuracy: 0.0000e+00
Epoch 437/500
1168/1168 [==============================] - 1s 882us/step - loss: 14732.4525 - accuracy: 0.0000e+00 - val_loss: 21610.3301 - val_accuracy: 0.0000e+000s - 

1168/1168 [==============================] - 1s 878us/step - loss: 13575.9552 - accuracy: 0.0000e+00 - val_loss: 21010.1738 - val_accuracy: 0.0000e+00
Epoch 482/500
1168/1168 [==============================] - 1s 816us/step - loss: 13519.6843 - accuracy: 0.0000e+00 - val_loss: 21114.2832 - val_accuracy: 0.0000e+00
Epoch 483/500
1168/1168 [==============================] - 1s 880us/step - loss: 12861.8214 - accuracy: 0.0000e+00 - val_loss: 22711.6270 - val_accuracy: 0.0000e+00
Epoch 484/500
1168/1168 [==============================] - 1s 878us/step - loss: 13504.0331 - accuracy: 0.0000e+00 - val_loss: 21870.3398 - val_accuracy: 0.0000e+00
Epoch 485/500
1168/1168 [==============================] - 1s 874us/step - loss: 13503.5620 - accuracy: 0.0000e+00 - val_loss: 21942.5488 - val_accuracy: 0.0000e+00
Epoch 486/500
1168/1168 [==============================] - 1s 882us/step - loss: 12785.5319 - accuracy: 0.0000e+00 - val_loss: 21344.0254 - val_accuracy: 0.0000e+00
Epoch 487/500
1168/1168 

### Make Prediction on test data

In [95]:
y_prediction = model.predict(final_test).flatten()

data = {"Id": df_test.Id,
        "SalePrice": y_prediction}

df_final = pd.DataFrame(data)
df_final.to_csv("final_sub.csv", index=False)


df_final.head()

,Id,SalePrice
0,1461,116139.710938
1,1462,146742.171875
2,1463,180887.187500
3,1464,186029.546875
4,1465,188178.781250
